Below is a Python program that simulates a hotel booking system for "The Best Inn". The program allows clients to book rooms, displays messages for available dates, and allows administrators to view all bookings.
This program requires the python-docx package to create a Word document for booking confirmations. Install it using: pip install python-docx
Explanation of the Program: 
Data Structure: The program uses a dictionary to keep track of room availability and bookings.
Client and Admin Options: Users can choose between being a client or an administrator. Clients can book rooms, while administrators can view all bookings.
Checking Availability: The program verifies if there are free rooms and finds nearby dates if the selected dates are fully booked.
Booking Confirmation: Upon successful booking, a confirmation document is generated and saved.
Password Protection for Admin: The admin section requires a password. After three failed attempts, it returns to the main menu.
Loading Bookings: The load_bookings method reads the existing bookings from a "bookings.docx" file in the "mybooking" directory at the program's start.
Saving Bookings: The save_booking method appends new bookings to the same "bookings.docx" file whenever a new reservation is created.
Directory Creation: A check is added to create the "mybooking" directory if it does not already exist.
Confirmation Document: Upon successful booking, a confirmation document is still created as before.
Directory Structure:
When you run this program, it will create a folder named "mybooking" in the same directory as the script. In this folder, you'll find the "bookings.docx" file containing information about bookings and individual confirmation files for each booking.
Running the Program:
Make sure you have the python-docx package installed and run the program. 
Input your desired booking options, and the program will handle storing and displaying booking information as specified!
When you run the program again, it will retain the previously made bookings.


In [ ]:
import datetime
import os
from collections import defaultdict
from docx import Document

class Hotel:
    def __init__(self):
        self.rooms = {
            'luxury': {'total': 5, 'price_per_night': 300, 'booked': defaultdict(list)},
            'double_suite': {'total': 25, 'price_per_night': 150, 'booked': defaultdict(list)},
            'single': {'total': 10, 'price_per_night': 75, 'booked': defaultdict(list)},
        }
        self.bookings = []  # Keep track of all bookings
        self.load_bookings()  # Load existing bookings from docx file

    def load_bookings(self):
        if not os.path.exists("mybooking"):
            os.makedirs("mybooking")
        
        booking_file = "mybooking/bookings.docx"
        if os.path.exists(booking_file):
            doc = Document(booking_file)
            for para in doc.paragraphs:
                booking_info = para.text.split(';')
                if len(booking_info) >= 5:  # Should at least have 5 pieces of info
                    email, check_in, check_out, room_class, number_of_rooms = booking_info
                    self.bookings.append({
                        'email': email,
                        'check_in': datetime.datetime.strptime(check_in, '%Y-%m-%d'),
                        'check_out': datetime.datetime.strptime(check_out, '%Y-%m-%d'),
                        'room_class': room_class,
                        'number_of_rooms': int(number_of_rooms),
                    })

    def save_booking(self, email, check_in, check_out, room_class, number_of_rooms):
        booking_file = "mybooking/bookings.docx"
        doc = Document()

        for booking in self.bookings:
            doc.add_paragraph(f"{booking['email']};{booking['check_in'].strftime('%Y-%m-%d')};"
                              f"{booking['check_out'].strftime('%Y-%m-%d')};{booking['room_class']};"
                              f"{booking['number_of_rooms']}")

        doc.add_paragraph(f"{email};{check_in.strftime('%Y-%m-%d')};{check_out.strftime('%Y-%m-%d')};{room_class};{number_of_rooms}")
        doc.save(booking_file)

    def check_availability(self, check_in, check_out, room_class, number_of_rooms):
        booked_dates = self.rooms[room_class]['booked']
        available = self.rooms[room_class]['total'] - len(booked_dates)
        for date in booked_dates:
            if check_in <= date <= check_out:
                available -= booked_dates[date].count(room_class)
        return available >= number_of_rooms

    def find_nearby_dates(self, check_in, check_out, room_class, number_of_rooms):
        near_dates = []
        delta = datetime.timedelta(days=1)
        current_check_in = check_in

        while not near_dates and current_check_in < check_in + datetime.timedelta(days=30):
            current_check_out = check_out + (current_check_in - check_in)
            if self.check_availability(current_check_in, current_check_out, room_class, number_of_rooms):
                near_dates.append((current_check_in, current_check_out))
            current_check_in += delta
            
        return near_dates

    def book_rooms(self, email, check_in, check_out, room_class, number_of_rooms):
        self.rooms[room_class]['booked'][check_in].extend([room_class] * number_of_rooms)
        self.bookings.append({
            'email': email,
            'check_in': check_in,
            'check_out': check_out,
            'room_class': room_class,
            'number_of_rooms': number_of_rooms,
        })
        self.save_booking(email, check_in, check_out, room_class, number_of_rooms)

    def calculate_price(self, room_class, number_of_rooms, number_of_nights):
        price = self.rooms[room_class]['price_per_night'] * number_of_rooms * number_of_nights
        return price

    def generate_confirmation(self, email, check_in, check_out, room_class):
        confirmation = Document()
        confirmation.add_paragraph(
            f"Your booking for the dates {check_in.strftime('%Y-%m-%d')} to {check_out.strftime('%Y-%m-%d')} "
            f"for a room class {room_class} is confirmed. Enjoy your stay."
        )
        confirmation_path = f'mybooking/booking_confirmation_{email}.docx'
        confirmation.save(confirmation_path)
        print(f"Booking confirmation sent to {email}.")

    def list_bookings(self):
        print("\nCurrent Bookings:")
        for idx, booking in enumerate(self.bookings, start=1):
            print(f"{idx}: {booking}")

def client_booking(hotel):
    while True:
        check_in = input("Enter check-in date (YYYY-MM-DD): ")
        check_out = input("Enter check-out date (YYYY-MM-DD): ")
        room_class = input("Choose room class (luxury, double_suite, single): ")
        number_of_rooms = int(input("Number of rooms: "))
        breakfast = input("Do you want breakfast? (yes or no): ").lower() == 'yes'

        check_in_date = datetime.datetime.strptime(check_in, '%Y-%m-%d')
        check_out_date = datetime.datetime.strptime(check_out, '%Y-%m-%d')

        if hotel.check_availability(check_in_date, check_out_date, room_class, number_of_rooms):
            total_price = hotel.calculate_price(room_class, number_of_rooms, (check_out_date - check_in_date).days)
            if breakfast:
                total_price += 15 * number_of_rooms * (check_out_date - check_in_date).days

            print(f"Total price for your booking: {total_price} USD")
            confirm = input("Do you want to book this? (yes or no): ").lower()
            if confirm == 'yes':
                email = input("Please enter your email address for booking confirmation: ")
                hotel.book_rooms(email, check_in_date, check_out_date, room_class, number_of_rooms)
                hotel.generate_confirmation(email, check_in_date, check_out_date, room_class)
                break
        else:
            available_dates = hotel.find_nearby_dates(check_in_date, check_out_date, room_class, number_of_rooms)
            if available_dates:
                nearest_check_in, nearest_check_out = available_dates[0]
                total_price = hotel.calculate_price(room_class, number_of_rooms, (nearest_check_out - nearest_check_in).days)
                print(f"We have open for booking rooms for these dates {nearest_check_in.strftime('%Y-%m-%d')} to {nearest_check_out.strftime('%Y-%m-%d')} for a price of {total_price} USD")
                confirm = input("Do you want to book this? (yes or no): ").lower()
                if confirm == 'yes':
                    email = input("Please enter your email address for booking confirmation: ")
                    hotel.book_rooms(email, nearest_check_in, nearest_check_out, room_class, number_of_rooms)
                    hotel.generate_confirmation(email, nearest_check_in, nearest_check_out, room_class)
                    break
            else:
                print("No available rooms found in the coming days.")

def admin_panel(hotel):
    attempts = 0
    while attempts < 3:
        password = input("Enter admin password: ")
        if password == "WeHaveTheBestHotel":
            hotel.list_bookings()
            return
        else:
            attempts += 1
            print("Incorrect password, try again.")
    
    print("Too many incorrect attempts. Returning to main menu.")

def main():
    hotel = Hotel()
    while True:
        user_type = input("Are you a client or an administrator? (client/admin): ").lower()
        if user_type == "client":
            client_booking(hotel)
        elif user_type == "admin":
            admin_panel(hotel)
        else:
            print("Invalid option selected. Please choose again.")

if __name__ == "__main__":
    main()